In [17]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch import autograd
from torch.utils import data
from torch.optim import Adam
from torchvision import datasets, transforms
import torchvision.models as models

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## prophet

In [19]:
DATASET_PATH = os.path.join('/content/drive/MyDrive/이어드림 머신러닝/22 02 16 - 고속도로 통행량/data')

In [20]:
df_train = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
df_train

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,20200517,19,311727,101285,10085,30637,10060,8749,148935,6801,26101,9370,74169,17764,34715,19806,42734,63382,14130,34866,71475,96849,79150,25109,14680,10465,128344,6726,15431,25597,14292,9300,22238,3786,16936,10729,20194
3275,20200517,20,305354,91426,8607,26021,8095,7198,136503,6147,20759,8432,68923,14593,30279,17634,37661,57815,13055,31943,65378,92116,72379,23165,11879,9135,131682,5501,15378,24661,14747,8239,20604,3203,15018,9767,17962
3276,20200517,21,306008,75113,6325,19933,5711,4494,129412,5134,14496,6739,54455,11997,27047,15943,29029,54627,12292,27424,60696,83628,57866,16629,8573,6488,127826,4216,12558,22781,14081,6392,17937,2447,12403,7825,14031
3277,20200517,22,237447,49498,4209,12145,3891,2718,96698,3526,9363,4543,35618,7996,19478,10010,18663,39694,8818,17770,45983,57321,35250,10761,5331,4109,94288,2578,8870,16640,11066,4427,11955,1495,7507,5387,8889


In [21]:
df_val = pd.read_csv(os.path.join(DATASET_PATH, 'validate.csv'))
df_val = df_val[168:]
df_val

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
168,20200518,0,82065,15172,1500,3294,1086,962,28931,1103,3014,1619,12596,3060,7383,2958,5829,11297,2890,5694,19553,18919,9907,2912,1884,1112,28189,618,2790,5147,4331,1329,3665,404,2242,1619,2314
169,20200518,1,51248,9840,813,2356,696,546,17888,720,1839,1502,7741,2132,4790,1743,4296,6781,1752,3651,12454,11849,6292,1750,1118,703,16683,430,1864,3269,2561,921,2081,272,1390,1003,1766
170,20200518,2,39026,7894,760,2413,408,549,13357,498,1360,1233,6475,2098,3965,1345,3957,4762,1250,3199,9640,9392,4961,1104,1004,485,11934,322,1313,2765,1931,920,1764,228,1136,922,1309
171,20200518,3,40993,10137,780,2701,420,741,15544,532,1671,1800,7720,2133,4083,1501,3992,6202,1505,4459,10802,11664,6015,1132,1061,851,13458,326,1766,3320,2060,892,2447,337,1495,975,1912
172,20200518,4,77863,19603,1276,5019,968,1160,32101,968,2574,2384,12018,3502,6430,2882,5957,11987,3176,9054,16490,23479,10676,1945,1520,1420,31698,669,2914,6986,3911,1368,4380,513,2940,1758,3629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
332,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
333,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
334,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [22]:
df_train = pd.concat([df_train, df_val]).reset_index()
df_train = df_train.drop(columns='index')
df_train

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,20200524,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3443,20200524,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3444,20200524,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3445,20200524,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [23]:
# df_train = df_train[168*8+85:].reset_index()
# df_train = df_train.drop(columns='index')
# df_train

In [24]:
# # len(df_train.columns[3:])
# drop_list = []
# for i in df_train.columns[3:]:
#     # drop_list.append(df_train.columns[]
#     drop_list.append(i)

# df_train = df_train.drop(columns=drop_list)
df_train['날짜'] = df_train['날짜'].astype('str')
df_train['시간'] = df_train['시간'].astype('str')
df_train['날짜'] = df_train['날짜'] + ' ' + df_train['시간']

In [25]:
df_train

,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200101 0,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,20200101 1,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,20200101 2,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,20200101 3,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,20200101 4,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,20200524 19,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3443,20200524 20,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3444,20200524 21,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3445,20200524 22,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [26]:
for i in range(len(df_train)):
    if len(df_train['시간'][i]) == 1:
        df_train['날짜'][i] = df_train['날짜'][i][0:4] + '-' + df_train['날짜'][i][4:6] + '-' + df_train['날짜'][i][6:8] + ' 0' + df_train['날짜'][i][9:10] + ':00:00'
    else:
        df_train['날짜'][i] = df_train['날짜'][i][0:4] + '-' + df_train['날짜'][i][4:6] + '-' + df_train['날짜'][i][6:8] + ' ' + df_train['날짜'][i][9:11] + ':00:00'

df_train['날짜'] = pd.to_datetime(df_train['날짜'])
df_train

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,날짜,시간,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,2020-01-01 00:00:00,0,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,2020-01-01 01:00:00,1,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,2020-01-01 02:00:00,2,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,2020-01-01 03:00:00,3,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,2020-01-01 04:00:00,4,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,2020-05-24 19:00:00,19,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3443,2020-05-24 20:00:00,20,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3444,2020-05-24 21:00:00,21,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3445,2020-05-24 22:00:00,22,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [27]:
df_train.dtypes

날짜      datetime64[ns]
시간              object
10               int64
100              int64
101              int64
120              int64
121              int64
140              int64
150              int64
160              int64
200              int64
201              int64
251              int64
270              int64
300              int64
301              int64
351              int64
352              int64
370              int64
400              int64
450              int64
500              int64
550              int64
600              int64
650              int64
652              int64
1000             int64
1020             int64
1040             int64
1100             int64
1200             int64
1510             int64
2510             int64
3000             int64
4510             int64
5510             int64
6000             int64
dtype: object

In [28]:
df_train = df_train.drop(columns='시간')
df_train

,날짜,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,2020-01-01 00:00:00,83247,19128,2611,5161,1588,892,32263,1636,7938,1894,14957,3527,11174,2930,8640,14175,2035,5231,18492,48520,15393,16628,8804,2790,38657,1311,3482,11299,7072,1176,3810,748,3920,2133,3799
1,2020-01-01 01:00:00,89309,19027,3337,5502,1650,1043,35609,1644,9896,1967,14682,3420,13884,3062,8349,15654,2417,5483,15283,57904,15834,19785,8483,3381,45759,1162,3849,13180,8771,1283,3763,782,3483,2057,4010
2,2020-01-01 02:00:00,66611,14710,2970,4631,1044,921,26821,1104,9343,1838,11120,3587,17141,3255,7025,12520,1859,5058,14327,53876,13866,24501,8167,2670,27425,768,2299,7986,5426,1536,3229,491,2634,1526,3388
3,2020-01-01 03:00:00,53290,13753,2270,4242,1021,790,21322,909,10009,1630,9836,3842,20715,2661,7226,9821,1724,4863,12787,51615,13528,28695,9459,2325,18893,632,1716,5703,3156,1104,2882,431,2488,1268,3686
4,2020-01-01 04:00:00,52095,17615,2406,3689,1840,922,22711,1354,15497,1264,10668,4559,28308,2894,8201,9038,960,5166,11479,47424,18156,26977,10944,3583,18135,875,2421,5816,2933,1206,2433,499,2952,1927,5608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3442,2020-05-24 19:00:00,314226,98345,10625,28618,8316,6684,141675,6619,23535,8572,67748,16791,33325,19124,42183,63229,13163,35383,73188,100849,84286,26458,15572,10341,139827,8254,16118,23304,14082,8447,21694,2180,15746,10903,21014
3443,2020-05-24 20:00:00,300001,87871,8226,22706,6981,5743,142933,6295,20953,7851,61609,14119,30216,18610,35556,59070,12788,30675,68255,95107,76441,24140,13340,8872,142469,5225,15297,21919,14526,7332,19732,1990,14096,10028,17787
3444,2020-05-24 21:00:00,304150,71126,6002,18317,4939,3779,133110,4781,15242,6101,48336,11534,25710,15651,27467,53787,12019,26317,61169,86434,61136,21076,9317,6392,138816,4072,12685,21135,14403,5443,16967,1359,11670,7963,14041
3445,2020-05-24 22:00:00,236751,44947,3575,11455,3135,2536,98582,3267,8849,4553,33607,8069,18293,9723,16602,38615,8703,17694,47609,58492,37501,12090,5901,3938,100767,2489,8093,14427,10914,3861,11397,859,7270,5194,8230


In [29]:
# !pip install pystan==2.19.1.1
# !pip install fbprophet==0.7.1
# !pip install prophet==1.0.1
from prophet import Prophet

In [30]:
# !pip install holidays
import holidays

holidays = holidays.KR()
holidays.get('2020-01-01')
holidays

{datetime.date(2020, 1, 1): "New Year's Day",
 datetime.date(2020, 1, 24): "The day preceding of Lunar New Year's Day",
 datetime.date(2020, 1, 25): "Lunar New Year's Day",
 datetime.date(2020, 1, 26): "The second day of Lunar New Year's Day",
 datetime.date(2020, 1, 27): "Alternative holiday of Lunar New Year's Day",
 datetime.date(2020, 3, 1): 'Independence Movement Day',
 datetime.date(2020, 4, 30): 'Birthday of the Buddha',
 datetime.date(2020, 5, 1): 'Labour Day',
 datetime.date(2020, 5, 5): "Children's Day",
 datetime.date(2020, 6, 6): 'Memorial Day',
 datetime.date(2020, 8, 15): 'Liberation Day',
 datetime.date(2020, 9, 30): 'The day preceding of Chuseok',
 datetime.date(2020, 10, 1): 'Chuseok',
 datetime.date(2020, 10, 2): 'The second day of Chuseok',
 datetime.date(2020, 10, 3): 'National Foundation Day',
 datetime.date(2020, 10, 9): 'Hangeul Day',
 datetime.date(2020, 10, 17): 'Alternative public holiday',
 datetime.date(2020, 12, 25): 'Christmas Day'}

In [31]:
df_holiday = pd.DataFrame(columns = ['ds', 'holiday'])

for i in range(len(holidays)):
    df_holiday['ds'] = list(holidays.keys())
    df_holiday['holiday'] = list(holidays.values())
df_holiday

,ds,holiday
0,2020-01-01,New Year's Day
1,2020-01-24,The day preceding of Lunar New Year's Day
2,2020-01-25,Lunar New Year's Day
3,2020-01-26,The second day of Lunar New Year's Day
4,2020-01-27,Alternative holiday of Lunar New Year's Day
5,2020-03-01,Independence Movement Day
6,2020-04-30,Birthday of the Buddha
7,2020-05-05,Children's Day
8,2020-05-01,Labour Day
9,2020-06-06,Memorial Day


In [33]:
# df_train.columns[0:5]
#  if df['mths_since_last_delinq{}'.format(i+1)][j] == 1:

submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)
submission_table

for col in df_train.columns[1:]:

    df_for = df_train[['날짜', '{}'.format(col)]]
    df_for.columns = ['ds', 'y']

    m = Prophet(holidays = df_holiday, weekly_seasonality = 13, seasonality_prior_scale= 13, holidays_prior_scale= 13)
    m.fit(df_for)

    # m = Prophet(holidays = df_holiday)
    # m.fit(df_for)

    future=m.make_future_dataframe(periods=24*7 , freq='H')  # 14일치를 예측할 것.
    forecast=m.predict(future)
    df_pred = forecast[['ds', 'yhat']][3615-168:].reset_index()
    
    submission_table['{}'.format(col)] = df_pred["yhat"]

submission_table

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=Tru

,timestamp,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200525_0,103370.667180,23813.711594,1688.820277,6020.159578,1608.822299,1412.941889,43642.313299,1519.515293,4301.086895,1891.990854,15466.267234,2862.772541,8366.180638,4680.815065,7827.549446,14670.510609,4662.304678,10013.784400,22494.070284,21280.971148,18973.695658,4302.958696,2508.938343,1819.044766,36395.159522,1270.452356,4357.191981,2526.008940,4688.782089,2065.798967,5825.065311,157.731566,3832.164053,3055.438353,4344.153167
1,20200525_1,75964.664482,19434.989498,1443.225143,5041.774843,1531.369973,1330.809163,32110.074357,1181.146180,3397.667615,1503.066924,12478.986977,1962.331813,6274.892136,3946.769062,5857.836288,10855.756544,3765.000232,8562.017031,15757.153995,15379.986730,15968.646547,3040.520472,2166.178107,1339.008345,28471.963833,1175.249404,3591.161902,1511.291455,3715.344357,1655.947338,5033.962971,276.669116,3849.782691,2848.011701,3939.233955
2,20200525_2,64336.540384,18514.699307,1535.960042,5296.869514,1406.826201,1365.732218,28467.135857,1046.071660,3275.764244,1676.179413,12065.584867,2165.212962,6360.336864,4057.855732,5763.369993,9732.781753,3513.300653,8609.993803,13748.976170,14234.087802,14994.974675,2830.621977,2193.587847,1285.221401,26616.688134,1122.564222,3252.551159,1297.218783,3279.759422,1725.187770,4988.913986,259.284041,3500.527916,2667.770650,3773.460153
3,20200525_3,77698.445271,23771.093469,2131.983845,6821.505811,1659.452342,1649.424473,34952.671176,1278.566255,4105.086601,2306.106082,14579.600837,3379.559063,8519.230923,5049.176115,7473.490170,12368.140036,4132.087270,10824.709856,16804.898055,20397.284168,17153.858850,3399.395501,2730.072729,1985.231224,36949.188266,1418.017549,4001.150805,4156.467211,4475.308004,2146.234880,5966.844797,308.751608,3873.377259,3125.573446,4778.005404
4,20200525_4,118887.394220,36361.892515,3293.119847,9396.359781,2773.484747,2310.549242,50899.924385,1996.606595,5796.249232,3143.088993,20632.664137,5091.201431,11680.008078,6808.016847,10585.477132,19408.041024,5628.374866,15337.796801,23360.081275,34725.790705,23563.028044,4311.145607,3823.187205,3447.952598,63321.133019,2384.134813,6381.207636,11540.435473,8013.156489,2689.126143,8194.331157,716.994217,6280.351144,4754.518753,7684.363977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,20200531_19,322960.227672,98797.185321,10175.483269,29687.255886,9650.979914,7667.689638,142537.895007,6871.461451,22583.781535,9145.619186,71238.164561,16547.528209,34012.822995,20264.376093,43586.701766,59620.125559,13162.134178,34125.321890,73415.076727,99798.635249,80878.357869,24818.022533,14991.983128,10118.585424,140523.950298,7775.967939,15913.904847,24051.346836,16076.692387,8722.081714,24192.846046,2982.992165,17075.787735,11398.214446,20677.459500
164,20200531_20,270209.474749,74558.087908,6997.874633,21988.523823,6403.049597,5332.128591,117971.283817,5221.856735,16781.066892,7060.501391,55399.524467,12238.193916,26736.226531,15348.214470,32847.215972,47144.876859,11038.640018,26793.573680,62523.025898,77486.595873,61732.004906,18919.409467,10167.812019,7076.466841,113009.749043,5187.457891,12748.010004,15593.998996,12800.754326,6828.611878,18464.914459,1661.653839,11984.944514,8101.933418,14432.592725
165,20200531_21,219882.281073,54048.198094,4542.723731,15820.294851,3763.278423,3419.957763,95946.836882,3784.738709,11909.190625,5296.629048,41022.660578,8816.203512,20800.914953,11224.433267,23693.355224,35461.137811,9092.033444,20497.110249,52117.454987,56890.366812,45068.185940,13901.462851,6579.767370,4731.677664,85265.706167,3123.384985,9597.062982,7761.642399,9554.379912,5250.162650,13480.915727,521.659186,7354.132035,5386.008892,9267.519293
166,20200531_22,175931.672688,39884.638361,3000.556592,11447.244496,2235.611193,2218.318697,765

In [34]:
submission_table.to_csv('prediction(train+val, w_seasonality13, seasonality_scale13, holidays_scale13).csv', index=False)

# for문 코드를 위한 사전시도 (skip)

In [22]:
# 날짜와 도로 1개를 남기고 ds와 y로 바꾸기

# len(df_train.columns[2:])
drop_list = []
for i in df_train.columns[2:]:
    # drop_list.append(df_train.columns[]
    drop_list.append(i)

df_train = df_train.drop(drop_list, axis = 1)
df_train.columns = ['ds', 'y']

df_train

,ds,y
0,2020-01-01 00:00:00,83247
1,2020-01-01 01:00:00,89309
2,2020-01-01 02:00:00,66611
3,2020-01-01 03:00:00,53290
4,2020-01-01 04:00:00,52095
...,...,...
3442,2020-05-24 19:00:00,314226
3443,2020-05-24 20:00:00,300001
3444,2020-05-24 21:00:00,304150
3445,2020-05-24 22:00:00,236751


prophet

In [26]:
# !pip install pystan==2.19.1.1
# !pip install fbprophet==0.7.1
# !pip install prophet==1.0.1
from prophet import Prophet

In [27]:
# !pip install holidays
import holidays

In [28]:
holidays = holidays.KR()
holidays.get('2020-01-01')
holidays

{datetime.date(2020, 1, 1): "New Year's Day",
 datetime.date(2020, 1, 24): "The day preceding of Lunar New Year's Day",
 datetime.date(2020, 1, 25): "Lunar New Year's Day",
 datetime.date(2020, 1, 26): "The second day of Lunar New Year's Day",
 datetime.date(2020, 1, 27): "Alternative holiday of Lunar New Year's Day",
 datetime.date(2020, 3, 1): 'Independence Movement Day',
 datetime.date(2020, 4, 30): 'Birthday of the Buddha',
 datetime.date(2020, 5, 1): 'Labour Day',
 datetime.date(2020, 5, 5): "Children's Day",
 datetime.date(2020, 6, 6): 'Memorial Day',
 datetime.date(2020, 8, 15): 'Liberation Day',
 datetime.date(2020, 9, 30): 'The day preceding of Chuseok',
 datetime.date(2020, 10, 1): 'Chuseok',
 datetime.date(2020, 10, 2): 'The second day of Chuseok',
 datetime.date(2020, 10, 3): 'National Foundation Day',
 datetime.date(2020, 10, 9): 'Hangeul Day',
 datetime.date(2020, 10, 17): 'Alternative public holiday',
 datetime.date(2020, 12, 25): 'Christmas Day'}

In [29]:
df_holiday = pd.DataFrame(columns = ['ds', 'holiday'])

for i in range(len(holidays)):
    df_holiday['ds'] = list(holidays.keys())
    df_holiday['holiday'] = list(holidays.values())
df_holiday

,ds,holiday
0,2020-01-01,New Year's Day
1,2020-01-24,The day preceding of Lunar New Year's Day
2,2020-01-25,Lunar New Year's Day
3,2020-01-26,The second day of Lunar New Year's Day
4,2020-01-27,Alternative holiday of Lunar New Year's Day
5,2020-03-01,Independence Movement Day
6,2020-04-30,Birthday of the Buddha
7,2020-05-05,Children's Day
8,2020-05-01,Labour Day
9,2020-06-06,Memorial Day


In [23]:
# prophet 학습

# m = Prophet()
# m.fit(df_train)

m = Prophet(holidays = df_holiday)
m.fit(df_train)
# m.train_holiday_names

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.


In [28]:
future=m.make_future_dataframe(periods=24*7 , freq='H')  # 14일치를 예측할 것.
forecast=m.predict(future)
df_pred = forecast[['ds', 'yhat']][3615-168:].reset_index()

In [29]:
df_pred

,index,ds,yhat
0,3447,2020-05-25 00:00:00,106617.039590
1,3448,2020-05-25 01:00:00,73495.844373
2,3449,2020-05-25 02:00:00,53873.486928
3,3450,2020-05-25 03:00:00,58775.563922
4,3451,2020-05-25 04:00:00,92875.743440
...,...,...,...
163,3610,2020-05-31 19:00:00,299636.118841
164,3611,2020-05-31 20:00:00,257778.632257
165,3612,2020-05-31 21:00:00,217092.613811
166,3613,2020-05-31 22:00:00,179693.233909


In [30]:
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y-%m-%d")
    end = datetime.strptime(end, "%Y-%m-%d")
    dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("2020-01-01", "2020-05-24")
# dates_val = date_range("2020-05-11", "2020-05-24")
# dates_test = date_range("2020-05-25", "2020-05-31")
print(dates)

['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01', '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05', '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09', '2020-03-10', '2020-03-11', '2020

In [31]:
for date in dates:
    if len(df_pred[df_pred['ds'] == date]) != 24:
        print(df_pred[df_pred['ds'] == date])

Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Index: []
Empty DataFrame
Columns: [index, ds, yhat]
Ind

In [44]:
df_pred['yhat']

0       91574.187698
1       58381.033788
2       39156.843443
3       46059.061167
4       83600.976513
           ...      
163    278177.570131
164    235524.761583
165    193221.302399
166    155019.079296
167    118972.169197
Name: yhat, Length: 168, dtype: float64

In [45]:
submission_file_path = os.path.join(DATASET_PATH, 'sample_submission.csv')
submission_table = pd.read_csv(submission_file_path)
submission_table

,timestamp,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200525_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20200525_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200525_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20200525_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200525_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,20200531_19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,20200531_20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,20200531_21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,20200531_22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
submission_table['10'] = df_pred["yhat"]
submission_table

,timestamp,10,100,101,120,121,140,150,160,200,201,251,270,300,301,351,352,370,400,450,500,550,600,650,652,1000,1020,1040,1100,1200,1510,2510,3000,4510,5510,6000
0,20200525_0,91574.187698,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20200525_1,58381.033788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20200525_2,39156.843443,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20200525_3,46059.061167,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20200525_4,83600.976513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,20200531_19,278177.570131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
164,20200531_20,235524.761583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
165,20200531_21,193221.302399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
166,20200531_22,155019.079296,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
